In [1]:
import torch
from torch.autograd import Function
import torchvision as tv
import torch.nn as nn
import torchvision.transforms as tvtf
import numpy as np
import torch.optim as optim
import torch.utils.data as data
from PIL import Image
import os

In [2]:
data_dir = 'C:\\Users\\black\\PycharmProjects\\AIML_Term_Paper\\mnist_m'
class MNISTMDataset(data.Dataset):
    def __init__(self, data_root, data_list, transform=None):
        self.root = data_root
        self.transform = transform

        f = open(data_list, 'r')
        data_list = f.readlines()
        f.close()

        self.n_data = len(data_list)

        self.img_paths = []
        self.img_labels = []

        for data in data_list:
            self.img_paths.append(data[:-3])
            self.img_labels.append(data[-2])

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item], self.img_labels[item]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels

    def __len__(self):
        return self.n_data

In [ ]:
image_size = 28
batch_size = 4

tf_source = tvtf.Compose([
    tvtf.Resize(image_size),
    tvtf.ToTensor(),
    tvtf.Normalize(mean=(0.1307,), std=(0.3081,))
])
tf_target = tvtf.Compose([
    tvtf.Resize(image_size),
    tvtf.ToTensor(),
    tvtf.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

ds_source = tv.datasets.MNIST(root=data_dir, train=True, transform=tf_source, download=True)
dl_source = torch.utils.data.DataLoader(ds_source, batch_size)

ds_target = MNISTMDataset(os.path.join(data_dir, 'mnist_m', 'mnist_m_train'),
                          os.path.join(data_dir, 'mnist_m', 'mnist_m_train_labels.txt'),
                         transform=tf_target)
dl_target = torch.utils.data.DataLoader(ds_target, batch_size)
ds_test = MNISTMDataset(os.path.join(data_dir, 'mnist_m', 'mnist_m_test'),
                          os.path.join(data_dir, 'mnist_m', 'mnist_m_test_labels.txt'),
                         transform=tf_target)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size)

In [3]:
class GradientReversalFunction(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        # grad_output is dL/dx (since our forward's output was x)
        # Backward pass is just to reverse the sign of the gradient
        # This will become the new dL/dx in the rest of the network
        output = ctx.alpha * grad_output.neg()
        # Must return number of inputs to forward()
        return output, None
image_size = 28
batch_size = 4

In [4]:


tf_source = tvtf.Compose([
    tvtf.Resize(image_size),
    tvtf.ToTensor(),
    tvtf.Normalize(mean=(0.1307,), std=(0.3081,))
])
tf_target = tvtf.Compose([
    tvtf.Resize(image_size),
    tvtf.ToTensor(),
    tvtf.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

ds_source = tv.datasets.MNIST(root=data_dir, train=True, transform=tf_source, download=True)
dl_source = torch.utils.data.DataLoader(ds_source, batch_size)

ds_target = MNISTMDataset(os.path.join(data_dir, 'mnist_m', 'mnist_m_train'),
                          os.path.join(data_dir, 'mnist_m', 'mnist_m_train_labels.txt'),
                         transform=tf_target)
dl_target = torch.utils.data.DataLoader(ds_target, batch_size)
ds_test = MNISTMDataset(os.path.join(data_dir, 'mnist_m', 'mnist_m_test'),
                          os.path.join(data_dir, 'mnist_m', 'mnist_m_test_labels.txt'),
                         transform=tf_target)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size)

In [5]:
class DANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.ReLU(True),
            nn.Conv2d(64, 50, kernel_size=5),
            nn.BatchNorm2d(50),
            nn.MaxPool2d(2),
            nn.ReLU(True),
            nn.Dropout2d(),
        )
        self.num_cnn_features = 50 * 4 * 4
        self.class_classifier = nn.Sequential(
            nn.Linear(self.num_cnn_features, 100),
            nn.BatchNorm1d(100), nn.Dropout2d(), nn.ReLU(True),
            nn.Linear(100, 100),
            nn.BatchNorm1d(100), nn.ReLU(True),
            nn.Linear(100, 10),
            nn.LogSoftmax(dim=1),
        )
        self.domain_classifier = nn.Sequential(
            nn.Linear(self.num_cnn_features, 100),
            nn.BatchNorm1d(100), nn.ReLU(True),
            nn.Linear(100, 2),
            nn.LogSoftmax(dim=1),
        )

    def forward(self, x, grl_lambda=1.0):
        x = x.expand(x.data.shape[0], 3, image_size, image_size)
        features = self.feature_extractor(x)
        features = features.view(-1, self.num_cnn_features)
        features_grl = GradientReversalFunction.apply(features, grl_lambda)
        class_pred = self.class_classifier(features)
        domain_pred = self.domain_classifier(features_grl)
        return class_pred, domain_pred

In [23]:
model = DANN()
lr = 1e-3
n_epochs = 30
# Setup optimizer as usual
optimizer = optim.Adam(model.parameters(), lr)
# Two losses functions this time
loss_fn_class = torch.nn.NLLLoss()
loss_fn_domain = torch.nn.NLLLoss()
batch_size = 256
dl_source = torch.utils.data.DataLoader(ds_source, batch_size)
dl_target = torch.utils.data.DataLoader(ds_target, batch_size)
# We'll train the same number of batches from both datasets
max_batches = min(len(dl_source), len(dl_target))
for epoch_idx in range(n_epochs):
    print(f'Epoch {epoch_idx+1:04d} / {n_epochs:04d}', end='\n=================\n')
    dl_source_iter = iter(dl_source)
    dl_target_iter = iter(dl_target)

    for batch_idx in range(max_batches):
        optimizer.zero_grad()
        # Training progress and GRL lambda
        p = float(batch_idx + epoch_idx * max_batches) / (n_epochs * max_batches)
        grl_lambda = 2. / (1. + np.exp(-10 * p)) - 1

        # Train on source domain
        X_s, y_s = next(dl_source_iter)
        y_s_domain = torch.zeros(y_s.size()[0], dtype=torch.long) # generate source domain labels

        class_pred, domain_pred = model(X_s, grl_lambda)
        loss_s_label = loss_fn_class(class_pred, y_s)
        loss_s_domain = loss_fn_domain(domain_pred, y_s_domain)

        # Train on target domain
        X_t, _ = next(dl_target_iter) # ignore target domain class labels
        y_t_domain = torch.ones(X_t.size()[0], dtype=torch.long) # generate target domain labels

        _, domain_pred = model(X_t, grl_lambda)
        loss_t_domain = loss_fn_domain(domain_pred, y_t_domain)

        loss = loss_t_domain + loss_s_domain + loss_s_label
        loss.backward()
        optimizer.step()

        print(f'[{batch_idx+1}/{max_batches}] '
              f'class_loss: {loss_s_label.item():.4f} ' f's_domain_loss: {loss_s_domain.item():.4f} '
              f't_domain_loss: {loss_t_domain.item():.4f} ' f'grl_lambda: {grl_lambda:.3f} '
             )

Epoch 0001 / 0030
[1/231] class_loss: 2.3596 s_domain_loss: 0.5102 t_domain_loss: 0.9626 grl_lambda: 0.000 
[2/231] class_loss: 2.2765 s_domain_loss: 0.5395 t_domain_loss: 0.9006 grl_lambda: 0.001 
[3/231] class_loss: 2.1381 s_domain_loss: 0.5701 t_domain_loss: 0.8441 grl_lambda: 0.001 
[4/231] class_loss: 2.0898 s_domain_loss: 0.6063 t_domain_loss: 0.7899 grl_lambda: 0.002 
[5/231] class_loss: 2.0654 s_domain_loss: 0.6382 t_domain_loss: 0.7592 grl_lambda: 0.003 
[6/231] class_loss: 1.9270 s_domain_loss: 0.6687 t_domain_loss: 0.7263 grl_lambda: 0.004 
[7/231] class_loss: 1.8278 s_domain_loss: 0.6981 t_domain_loss: 0.6937 grl_lambda: 0.004 
[8/231] class_loss: 1.7908 s_domain_loss: 0.7169 t_domain_loss: 0.6672 grl_lambda: 0.005 
[9/231] class_loss: 1.7492 s_domain_loss: 0.7331 t_domain_loss: 0.6528 grl_lambda: 0.006 
[10/231] class_loss: 1.7060 s_domain_loss: 0.7423 t_domain_loss: 0.6493 grl_lambda: 0.006 
[11/231] class_loss: 1.6330 s_domain_loss: 0.7393 t_domain_loss: 0.6525 grl_lambd

In [ ]:
ds_test = MNISTMDataset(os.path.join(data_dir, 'mnist_m', 'mnist_m_test'),
                          os.path.join(data_dir, 'mnist_m', 'mnist_m_test_labels.txt'),
                         transform=tf_target)
dl_test = torch.utils.data.DataLoader(ds_test, 4)

In [22]:
def calculate_accuracy(model, dataloader):
    correct = 0
    total = 0
    print(len(dataloader.dataset))
    with torch.no_grad():
        for images, labels in dataloader:
            print(images.size())
            outputs, _ = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print(100*correct/total,total)

    accuracy = 100 * correct / total
    return accuracy
calculate_accuracy(model,dl_test)

9001
torch.Size([4, 3, 28, 28])
100.0 4
torch.Size([4, 3, 28, 28])
75.0 8
torch.Size([4, 3, 28, 28])
75.0 12
torch.Size([4, 3, 28, 28])
75.0 16
torch.Size([4, 3, 28, 28])
70.0 20
torch.Size([4, 3, 28, 28])
66.66666666666667 24
torch.Size([4, 3, 28, 28])
64.28571428571429 28
torch.Size([4, 3, 28, 28])
65.625 32
torch.Size([4, 3, 28, 28])
63.888888888888886 36
torch.Size([4, 3, 28, 28])
60.0 40
torch.Size([4, 3, 28, 28])
61.36363636363637 44
torch.Size([4, 3, 28, 28])
58.333333333333336 48
torch.Size([4, 3, 28, 28])
57.69230769230769 52
torch.Size([4, 3, 28, 28])
55.357142857142854 56
torch.Size([4, 3, 28, 28])
56.666666666666664 60
torch.Size([4, 3, 28, 28])
56.25 64
torch.Size([4, 3, 28, 28])
55.88235294117647 68
torch.Size([4, 3, 28, 28])
56.94444444444444 72
torch.Size([4, 3, 28, 28])
55.26315789473684 76
torch.Size([4, 3, 28, 28])
56.25 80
torch.Size([4, 3, 28, 28])
57.142857142857146 84
torch.Size([4, 3, 28, 28])
59.09090909090909 88
torch.Size([4, 3, 28, 28])
57.608695652173914 92

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 100])